In [1]:
%matplotlib inline
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

### 使用在ImageNet数据集上预训练的ResNet-18模型来提取图像特征

In [2]:
# 加载了预训练的 ResNet-18 模型，并显示了模型的最后三个子模块。
pretrained_net=torchvision.models.resnet18(pretrained=True)
list(pretrained_net.children())[-3:]

C:\Users\eyrine\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\eyrine\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\eyrine/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


[Sequential(
   (0): BasicBlock(
     (conv1): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (downsample): Sequential(
       (0): Conv2d(256, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
       (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
   )
   (1): BasicBlock(
     (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): Batc

In [5]:
#不要最后两层
net=nn.Sequential(*list(pretrained_net.children())[:-2])
                  
X = torch.rand(size=(1,3,320,480))
net(X).shape

torch.Size([1, 512, 10, 15])

### 使用1x1卷积层将输出通道数转换为PascalVOC2012数据集的类数（21类）。将要素地图的高宽和宽度增加32倍。

In [ ]:
num_classed=21
net.add_module('final_conv',nn.Conv2d(512,num_classes,kernel_size=1))
net.add_module('transpose_conv',
               nn.ConvTranspose2d(
                   num_classes,num_classes,
                   kernel_size=64,padding=16,
                   stride=32))

### 初始化转置卷积层

双线性插值（bilinear interpolation） 是常用的上采样方法之一，它也经常用于初始化转置卷积层。
为了解释双线性插值，假设给定输入图像，我们想要计算上采样输出图像上的每个像素。

1.将输出图像的坐标(x,y)映射到输入图像的坐标(x',y')上。 例如，根据输入与输出的尺寸之比来映射。 请注意，映射后的x'和y'是实数。
2.在输入图像上找到离坐标(x',y')最近的4个像素。
3.输出图像在坐标(x,y)上的像素依据输入图像上这4个像素及其与(x',y')的相对距离来计算。

双线性插值的上采样可以通过转置卷积层实现，内核由以下bilinear_kernel函数构造。

In [6]:
def bilinear_kernel(in_channels,out_channels,kernel_size):
    factor=(kernel_size+1)//2
    if kernel_sizs%2==1:
        center=factor-1
    else:
        center=factor-0.5
    og=(torch.arrange(kernel_size).reshape(-1,1),
        torch.arange(kernel_size).reshape(1,-1))
    filt=(1-torch.abs(og[0]-center)/factor)*\
         (1-torch.abs(og[1]-center)/factor)
    weight=torch.zeros((in_channels,out_channels,
                        kernel_size,kernel_size))
    weight=torch.zeros((in_channels,out_channels,
                        kernel_size,kernel_size))
    weight[range(in_channels),range(out_channels),:,:]=filt
    return weight;